#Training StyleGAN2 on Colab
Y’all won’t stop asking me about this so here ya go 😂

If it were me I’d sign up for Colab Pro ($10/month) to get a couple extra hours of training time in per session. But you crazy kids can do whatever you want.

##Mounting Google Drive
So I’m actually gonna install my entire repo directly into Google Drive. This will make the setup a little easier, but its a little strange I admit.

First, connect your Drive to Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Install the repo
**Only do this for the first time ever setting this up!**

If this is your first time ever running this notebook, you’ll want to install my fork of StyleGAN2 to your Drive account. Make sure you have ample space on your Drive (I’d say at least 50GB). This will install the repo and add some dependecies (like transferring from FFHQ the first time).

Every time after your first use of this notebook you’ll want to skip this cell and run the cell after this.

In [ ]:
#SKIP this if you already have a stylegan2 folder in your google drive
%cd /content/drive/My\ Drive/
!mkdir stylegan2-colab
%cd stylegan2-colab/
!git clone https://github.com/dvschultz/stylegan2
%cd stylegan2
!mkdir pkl
%cd pkl
!gdown --id 1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF #inception: https://drive.google.com/open?id=1JLqXE5bGZnlu2BkbLPD5_ZxoO3Nz-AvF
%cd ../
!mkdir results
!mkdir results/00001-pretrained
%cd results/00001-pretrained
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_
!mv stylegan2-ffhq-config-f.pkl network-snapshot-10000.pkl
%cd ../../
%mkdir datasets

/content/drive/My Drive
mkdir: cannot create directory ‘stylegan2-colab’: File exists
/content/drive/My Drive/stylegan2-colab
fatal: destination path 'stylegan2' already exists and is not an empty directory.
/content/drive/My Drive/stylegan2-colab/stylegan2
mkdir: cannot create directory ‘pkl’: File exists
/content/drive/My Drive/stylegan2-colab/stylegan2/pkl
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <m

##Picking up from a previous session
If you already have the StyleGAN2 repo installed in Drive skip the above cell and run the following. This will make sure you have the latest version in case I make updates.

In [ ]:
#USE this if you already have a stylegan2 folder in google drive
%cd /content/drive/My\ Drive/stylegan2-colab/stylegan2
!git pull

/content/drive/My Drive/stylegan2-colab/stylegan2
^C


##Make sure Tensorflow 1.15 is set
Colab now defaults to Tensorflow 2. Make sure you run this cell to reset it to TF1.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


##Converting your dataset
StyleGAN requires you to convert your standard jpg or png images into a new format (.tfrecords). 

I’ve seen some recommendations to run this command every time you restart your Colab machine. I think if you ahve a small-ish dataset (< 2000 images) that’s probably unnecessary.

I recommend you upload your dataset to Google Drive and copy its path from the Drive folder in Colab and paste its path in the below cell.

After the `create_from_images` argument you need to pass in two paths. The first path is where the .tfrecords files should be output (just edit the last part to have a unique name). The second path is to the directory of your images.




In [ ]:
#2nd argument is where to put your tfrecords files
#3rd should point at your image dataset
!python dataset_tool.py create_from_images /content/drive/MyDrive/outputgan /content/drive/MyDrive/0001_512

Loading images from "/content/drive/MyDrive/0001_512"
  0% 0/47 [00:00<?, ?it/s]dataset_tool.py:87: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
100% 47/47 [00:28<00:00,  1.67it/s]


##Training
Note: this will require you to restart your Colab machine every 8–16 hours. You’ve been warned!

This library is set up to automatically find your latest .pkl file so it should keep re-training and making progress.

##Training Options
`--dataset`

This should be the name you used in the first path when converting your dataset.

`--mirror-augment`

Using this option loads some images at random mirrored horizontally (left-to-right). This might help if you have a very small dataset.

`--metrics`

METRICS DON’T MATTER. It’s art! Use your eyes. Set `--metrics=None` and live your life.

If you must use metrics, you have a few options. `fid50k`, the default, uses Frechet Inception Distance score. It’s what was used in StyleGAN1 and what most people know. It’s fine for images of animals and things, but it’s not great. `ppl_wend` is what StyleGAN2 prefers and claims to be more accurate. There are a bunch of other options but I’d recommend you stick with those. Note that both of these take 30–45minutes to run every time it runs so that cuts into your training time in Colab.


In [ ]:
!python run_training.py --num-gpus=1 --data-dir=./datasets --config=config-f --dataset=outputgan --mirror-augment=true --metrics=None

Local submit - run_dir: results/00018-stylegan2-outputgan-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x755a000 @  0x7f00f0ff3001 0x7f00edabb1af 0x7f00edb11c23 0x7f00edb12a87 0x7f00edbb4823 0x5936cc 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x532b86 0x53786a 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x593dd7 0x5118f8 0x593dd7 0x511e2c 0x549576 0x4bca8a 0x5134a6 0x549576 0x4bca8a 0x5134a6 0x593dd7
tcmalloc: large alloc 4294967296 bytes == 0x7eff266d0000 @  0x7f00f0ff11e7 0x7f00edabb0ce 0x7f00edb11cf5 0x7f00edb11f4f 0x7f00edbb4673 0x5936cc 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e
tcmalloc: large alloc 4294967296 bytes == 0x7efe256ce000 @  0x7f00f

In [ ]:
!pip3 install opensimplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Once running, your training files will show up in the results folder.

#Testing the model (generating images)
The following command will generate 55 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

`--truncation-psi`

Truncation is a special argument of StyleGAN. Essentially values that are closer to 0 will be more real than numbers further away from 0. I generally recommend a value between `0.5` and `1.0`. `0.5` will give you pretty "realistic" results, while `1.0` is likely to give you "weirder" results.

In [ ]:
!python run_generator.py generate-images --network=results/00018-stylegan2-outputgan-1gpu-config-f/submit_config.pkl --seeds=3875451-3876000 --truncation-psi=0.7

Local submit - run_dir: results/00019-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "results/00018-stylegan2-outputgan-1gpu-config-f/submit_config.pkl"...
Traceback (most recent call last):
  File "run_generator.py", line 490, in <module>
    main()
  File "run_generator.py", line 485, in main
    dnnlib.submit_run(sc, func_name_map[subcmd], **kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 343, in submit_run
    return farm.submit(submit_config, host_run_dir)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/internal/local.py", line 22, in submit
    return run_wrapper(submit_config)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 280, in run_wrapper
    run_func_obj(**submit_config.run_func_kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/run_generator.py", line 120, in generate_images
    _G, 

Let’s zip the generated files and download them.

In [ ]:
!zip -r generated-0.7.zip /content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images

  adding: content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images/ (stored 0%)
  adding: content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images/submit_config.pkl (deflated 43%)
  adding: content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images/submit_config.txt (deflated 53%)
  adding: content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images/log.txt (deflated 60%)
  adding: content/drive/MyDrive/stylegan2-colab/stylegan2/results/00019-generate-images/_finished.txt (stored 0%)


##Interpolation


In [ ]:
!python run_generator.py generate-latent-walk --network=/content/drive/MyDrive/stylegan2-colab/stylegan2/results/00018-stylegan2-outputgan-1gpu-config-f/submit_config.pkl --seeds=3,11,17,25,3 --frames 200 --truncation-psi=0.7

Local submit - run_dir: results/00020-generate-latent-walk
dnnlib: Running run_generator.generate_latent_walk() on localhost...
Loading networks from "/content/drive/MyDrive/stylegan2-colab/stylegan2/results/00018-stylegan2-outputgan-1gpu-config-f/submit_config.pkl"...
Traceback (most recent call last):
  File "run_generator.py", line 490, in <module>
    main()
  File "run_generator.py", line 485, in main
    dnnlib.submit_run(sc, func_name_map[subcmd], **kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 343, in submit_run
    return farm.submit(submit_config, host_run_dir)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/internal/local.py", line 22, in submit
    return run_wrapper(submit_config)
  File "/content/drive/MyDrive/stylegan2-colab/stylegan2/dnnlib/submission/submit.py", line 280, in run_wrapper
    run_func_obj(**submit_config.run_func_kwargs)
  File "/content/drive/MyDrive/stylegan2-colab/stylega

In [ ]:
#convert to video 
!ffmpeg -r 24 -i ./results/00001-generate-latent-walk/step%05d.png -vcodec libx264 -pix_fmt yuv420p latent-walk-v2.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
rm -r /content/drive/My Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f

rm: cannot remove '/content/drive/My': No such file or directory
rm: cannot remove 'Drive/stylegan2-colab-test/stylegan2/results/00002-stylegan2-birdaus-1gpu-config-f': No such file or directory
